In [1]:
# Импорт библиотек
import os
import time
import cotel
import GTU
import mat_properties as prop
import numpy as n
import pandas as pd
from scipy.optimize import root

# Основные константы
calctolerance = 10**-2
calcmethod = "hybr"
KPDnasos = 0.8074
KPD = 0.99

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streamsZAD = pd.read_excel(
    "streams.xlsx", sheet_name="water", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)


# Состав газов в номинале
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
os.environ["RPPREFIX"] = r"C:/Program Files (x86)/REFPROP"
fractiongas0 = (
    gas_streams0.at["GTU-PEVD", "N2"],
    gas_streams0.at["GTU-PEVD", "O2"],
    gas_streams0.at["GTU-PEVD", "CO2"],
    gas_streams0.at["GTU-PEVD", "H2O"],
    gas_streams0.at["GTU-PEVD", "Ar"],
)

gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)


# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure= gas_streams0.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures))
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = Enthalpies


####################################################################################################
# Расчет ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, GTU_input, "GTU-KU")
Gas_turbine_res = Gas_turbine.calc()
electric.loc["GTU", "N":"KPD"] = [Gas_turbine_res["N"], Gas_turbine_res["eff"]]
electric.loc["DK", "N"] = Gas_turbine_res["Ndk"]
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
gas_streams.loc["GTU-KU", "T"] = Gas_turbine_res["T"]
gas_streams.loc["GTU-KU", "G"] = Gas_turbine_res["G"]
gas_streams.loc["GTU-KU", "P"] = 0.1
Gas_turbine_composition=pd.read_excel("input.xlsx", sheet_name="Gas_composition0", index_col=0)

# чтение с вводного экселя входных данных
Pvd = water_streamsZAD.loc["IVD-PEVD", "P"]
Pnd = water_streamsZAD.loc["IND-PPND", "P"]
Twat0 = water_streams0.loc["REC-GPK", "T"]

#Параметры газа на выходе в КУ
gas_streams.loc["GTU-PEVD", "T":"P"] = gas_streams.loc["GTU-KU", "T":"P"]
gas_streams.loc["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
gas_streams.loc["GTU-PEVD", "N2":"Ar"]=Gas_turbine_composition.loc["Fraction", "N2":"Ar"]

# Состав газов при частичной нагрузке
fractiongas = (
    gas_streams.at["GTU-PEVD", "N2"],
    gas_streams.at["GTU-PEVD", "O2"],
    gas_streams.at["GTU-PEVD", "CO2"],
    gas_streams.at["GTU-PEVD", "H2O"],
    gas_streams.at["GTU-PEVD", "Ar"],
)
gas1 = prop.Materials_prop(
    gasmix,
    fractiongas,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)

# Задание энтальпий газа в рабочем режиме
Temperatures = gas_streams.loc["GTU-KU":"GPK-out", "T"]
Pressure= gas_streams.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas1.p_t(Pressure, x)["h"], Temperatures))
gas_streams.loc["GTU-KU":"GPK-out", "H"] = Enthalpies

# Задание в раб табл параметров по воде
water_streams.loc["IVD-PEVD":"PEN-EVD", "P"] = Pvd
water_streams.loc["IND-PPND":"REC-GPK", "P"] = Pnd
water_streams.loc["BND-PEN", "P"] = Pnd
water_streams.loc["REC-GPK", "T"] = Twat0

# Расчёт параметров на насыщении в испарительных поверхностях (в 1 итерации приближение: недогрев до н.п. на входе 3°С)
water_streams.loc["IVD-PEVD", "T"] = water.p_q(Pvd, 1)["T"]
water_streams.loc["IVD-PEVD", "H"] = water.p_q(Pvd, 1)["h"]
water_streams.loc["IND-PPND", "T"] = water.p_q(Pnd, 1)["T"]
water_streams.loc["IND-PPND", "H"] = water.p_q(Pnd, 1)["h"]
water_streams.loc["EVD-IVD", "T"] = water.p_q(Pvd, 1)["T"] - 1
water_streams.loc["GPK-IND", "T"] = water.p_q(Pnd, 1)["T"] - 1
water_streams.loc["EVD-IVD", "H"] = water.p_t(Pvd, water_streams.loc["EVD-IVD", "T"])[
    "h"
]
water_streams.loc["GPK-IND", "H"] = water.p_t(Pnd, water_streams.loc["GPK-IND", "T"])[
    "h"
]

####################################################################################################
# расчет котла целиком
Whole_cotel = cotel.cotel_all(
    KPD,
    KPDnasos,
    calctolerance,
    gas0,
    gas1,
    water,
    calcmethod,
    gas_streams0,
    water_streams0,
    gas_streams,
    water_streams,
)
Whole_cotel.calc(maxiterations=50)

print(
    "Степень сухости пара в ЭВД: ",
    water.p_h(water_streams.at["EVD-IVD", "P"],
              water_streams.at["EVD-IVD", "H"])["Q"],
)
print(
    "Степень сухости пара в ГПК: ",
    water.p_h(water_streams.at["GPK-IND", "P"],
              water_streams.at["GPK-IND", "H"])["Q"],
)

Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.4 сек. ---
IVD:--- 0.5 сек. ---
Prop:--- 0.1 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 1.5 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.3306819071210305
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.4 сек. ---
IVD:--- 0.6 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 1.5 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.689835385600812
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.5 сек. ---
IVD:--- 0.6 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.6 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 1.7 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.39244188234085087
Prop:--- 0.0 сек. ---
Iter:--- 0.3 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.4 сек. ---
IVD:--- 0.5 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 1.4 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.34517916623971456
Prop:--- 0.0 сек. ---
Iter:--- 0.3 сек. ---
Else:--- 0.0 сек. -

In [2]:
# Вывод газовых потоков
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-KU,542.010558,0.1,958.533980,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
GTU-PEVD,542.010558,0.1,958.533980,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,467.009396,0.1,872.853146,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,299.502541,0.1,686.847010,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,231.061108,0.1,612.884827,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,228.479437,0.1,610.115788,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,174.810717,0.1,552.862790,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,102.949560,0.1,477.005293,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065


In [3]:
# Вывод водяных потоков
water_streams['PEVD-DROSVD':'REC-GPK']

,T,P,H,G,S
PEVD-DROSVD,509.964938,8.405662,3419.417022,63.527299,NaN
IVD-PEVD,301.297556,8.746000,2747.104023,63.527299,NaN
EVD-IVD,289.724525,8.746000,1287.454403,63.527299,NaN
PEN-EVD,166.221575,8.746000,707.115426,63.527299,NaN
BND-PEN,164.738361,0.696400,696.095567,63.527299,NaN
PPND-DROSND,209.634008,0.692081,2867.097145,13.166354,NaN
IND-PPND,164.738361,0.696400,2762.534980,13.166354,NaN
GPK-IND,160.739889,0.696400,678.727342,76.693653,NaN
GPK-REC,160.739889,0.696400,678.727342,88.557319,NaN
REC-GPK,60.000000,0.696400,251.748396,88.557219,NaN


In [4]:
water_streams0['PEVD-DROSVD':'REC-GPK']

,T,P,H,G,S
PEVD-DROSVD,511.500000,8.4070,3423.209978,63.40,NaN
IVD-PEVD,301.297556,8.7460,2747.104023,63.40,NaN
EVD-IVD,298.800000,8.7460,1337.948265,63.40,NaN
PEN-EVD,166.500000,8.7460,708.319849,63.40,NaN
BND-PEN,164.738361,0.6964,696.095567,63.40,NaN
PPND-DROSND,211.600000,0.6910,2871.481837,14.76,NaN
IND-PPND,164.738361,0.6964,2762.534980,14.76,NaN
GPK-IND,164.600000,0.6964,695.493701,78.16,NaN
GPK-REC,164.600000,0.6964,695.493701,82.91,NaN
REC-GPK,60.000000,0.6964,251.748396,82.91,NaN


In [5]:
Errors_water_T= abs(water_streams['PEVD-DROSVD':'REC-GPK']['T'] -water_streams0['PEVD-DROSVD':'REC-GPK']['T'])/water_streams0['PEVD-DROSVD':'REC-GPK']['T']*100
Errors_water_G= abs(water_streams['PEVD-DROSVD':'REC-GPK']['G'] -water_streams0['PEVD-DROSVD':'REC-GPK']['G'])/water_streams0['PEVD-DROSVD':'REC-GPK']['G']*100

Errors_water_G

PEVD-DROSVD     0.200787
IVD-PEVD        0.200787
EVD-IVD         0.200787
PEN-EVD         0.200787
BND-PEN         0.200787
PPND-DROSND    10.797061
IND-PPND       10.797061
GPK-IND         1.876084
GPK-REC         6.811384
REC-GPK         6.811263
Name: G, dtype: float64

In [6]:
# Невязка по всем параметрам
Qgas = (
    KPD
    * gas_streams.at["GTU-PEVD", "G"]
    * (gas_streams.at["GTU-PEVD", "H"] - gas_streams.at["PEVD-IVD", "H"])
)
Qwat = water_streams.at["PEVD-DROSVD", "G"] * (
    water_streams.at["PEVD-DROSVD", "H"] - water_streams.at["IVD-PEVD", "H"]
)
print("dQ/Q ПЕВД+ИВД", ((Qgas - Qwat) / Qgas * 100))

Qgas1 = (
    KPD
    * gas_streams.at["GTU-PEVD", "G"]
    * (gas_streams.at["GTU-PEVD", "H"] - gas_streams.at["IVD-EVD", "H"])
)
Qwat1 = water_streams.at["PEVD-DROSVD", "G"] * (
    water_streams.at["PEVD-DROSVD", "H"] - water_streams.at["EVD-IVD", "H"]
)
print("dQ/Q ПЕВД+ИВД+ЭВД", ((Qgas1 - Qwat1) / Qgas1 * 100))

Qgas = (
    KPD
    * gas_streams.at["EVD-PPND", "G"]
    * (gas_streams.at["EVD-PPND", "H"] - gas_streams.at["IND-GPK", "H"])
)
Qwat = (
    water_streams.at["IND-PPND", "G"]
    * (water_streams.at["IND-PPND", "H"] - water_streams.at["GPK-IND", "H"])
    + water_streams.at["PPND-DROSND", "G"]
    * (water_streams.at["PPND-DROSND", "H"] - water_streams.at["IND-PPND", "H"])
    + water_streams.at["BND-PEN", "G"]
    * (water_streams.at["BND-PEN", "H"] - water_streams.at["GPK-IND", "H"])
)
print("dQ/Q ППНД+ИНД", ((Qgas - Qwat) / Qgas * 100))

Qgas1 = (
    KPD
    * gas_streams.at["EVD-PPND", "G"]
    * (gas_streams.at["EVD-PPND", "H"] - gas_streams.at["GPK-out", "H"])
)
Qwat1 = (
    water_streams.at["GPK-IND", "G"]
    * (water_streams.at["GPK-IND", "H"] - water_streams.at["X-GPK", "H"])
    + water_streams.at["IND-PPND", "G"]
    * (water_streams.at["IND-PPND", "H"] - water_streams.at["GPK-IND", "H"])
    + water_streams.at["PPND-DROSND", "G"]
    * (water_streams.at["PPND-DROSND", "H"] - water_streams.at["IND-PPND", "H"])
    + water_streams.at["BND-PEN", "G"]
    * (water_streams.at["BND-PEN", "H"] - water_streams.at["GPK-IND", "H"])
)
print("dQ/Q ППНД+ИНД+ГПК", (Qgas1 - Qwat1) / Qgas1 * 100)

Qgasall = (
    KPD
    * gas_streams.at["GTU-PEVD", "G"]
    * (gas_streams.at["GTU-PEVD", "H"] - gas_streams.at["GPK-out", "H"])
)
Qwatall = (
    water_streams.at["PPND-DROSND", "G"]
    * (water_streams.at["PPND-DROSND", "H"] - water_streams.at["X-GPK", "H"])
    + water_streams.at["PEVD-DROSVD", "G"]
    * (water_streams.at["PEVD-DROSVD", "H"] - water_streams.at["X-GPK", "H"])
    - water_streams.at["BND-PEN", "G"]
    * (water_streams.at["PEN-EVD", "H"] - water_streams.at["BND-PEN", "H"])
)
print("dQ/Qsumm", (Qgasall - Qwatall) / Qgasall * 100)

dQ/Q ПЕВД+ИВД -0.003637410014117608
dQ/Q ПЕВД+ИВД+ЭВД -0.008922866323422889
dQ/Q ППНД+ИНД 0.008430707692408217


KeyError: 'X-GPK'

In [ ]:
gas_streams0

In [ ]:
water_streams0

In [ ]:
electric

In [ ]:
heaters

In [ ]:
GTU_input